In [1]:

import torch
import os

from pytorch_adapt.adapters import DANN, MCD, VADA, CDAN, RTN, ADDA, Aligner, SymNets
from pytorch_adapt.containers import Models, Optimizers, LRSchedulers
from pytorch_adapt.models import Discriminator,  office31C, office31G
from pytorch_adapt.containers import Misc
from pytorch_adapt.layers import RandomizedDotProduct
from pytorch_adapt.layers import MultipleModels, CORALLoss, MMDLoss
from pytorch_adapt.utils import common_functions
from pytorch_adapt.containers import LRSchedulers

from classifier_adapter import ClassifierAdapter

from utils import HP, DAModels

import copy

import matplotlib.pyplot as plt
import torch
import os
import gc
from datetime import datetime

from pytorch_adapt.datasets import DataloaderCreator, get_office31
from pytorch_adapt.frameworks.ignite import CheckpointFnCreator, Ignite
from pytorch_adapt.validators import AccuracyValidator, IMValidator, ScoreHistory, DiversityValidator, EntropyValidator, MultipleValidators

from models import get_model
from utils import DAModels

from vis_hook import VizHook
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

device(type='cuda', index=0)

In [2]:
import argparse
parser = argparse.ArgumentParser()
parser.add_argument('--max_epochs', default=1, type=int)
parser.add_argument('--patience', default=2, type=int)
parser.add_argument('--batch_size', default=64, type=int)
parser.add_argument('--num_workers', default=1, type=int)
parser.add_argument('--trials_count', default=1, type=int)
parser.add_argument('--initial_trial', default=0, type=int)
parser.add_argument('--download', default=False, type=bool)
parser.add_argument('--root', default="./")
parser.add_argument('--data_root', default="./datasets/pytorch-adapt/")
parser.add_argument('--results_root', default="./results/")
parser.add_argument('--model_names', default=["DANN"], nargs='+')
parser.add_argument('--lr', default=0.0001, type=float)
parser.add_argument('--gamma', default=0.99, type=float)
parser.add_argument('--hp_tune', default=False, type=bool)
parser.add_argument('--source', default=None)
parser.add_argument('--target', default=None) 
parser.add_argument('--vishook_frequency', default=5, type=int)
    

args = parser.parse_args("")
print(args)


Namespace(batch_size=64, data_root='./datasets/pytorch-adapt/', download=False, gamma=0.99, hp_tune=False, initial_trial=0, lr=0.0001, max_epochs=1, model_names=['DANN'], num_workers=1, patience=2, results_root='./results/', root='./', source=None, target=None, trials_count=1, vishook_frequency=5)


In [45]:
source_domain = 'amazon'
target_domain = 'webcam'
datasets = get_office31([source_domain], [],
                            folder=args.data_root,
                            return_target_with_labels=True,
                            download=args.download)

dc = DataloaderCreator(batch_size=args.batch_size,
                    num_workers=args.num_workers,
                    )

weights_root = os.path.join(args.data_root, "weights")

G = office31G(pretrained=True, model_dir=weights_root).to(device)
C = office31C(domain=source_domain, pretrained=True,
                model_dir=weights_root).to(device)


optimizers = Optimizers((torch.optim.Adam, {"lr": 1e-4}))
lr_schedulers = LRSchedulers((torch.optim.lr_scheduler.ExponentialLR, {"gamma": 0.99}))    

models = Models({"G": G, "C": C})
adapter= ClassifierAdapter(models=models, optimizers=optimizers, lr_schedulers=lr_schedulers)

In [ ]:

output_dir = "tmp"
checkpoint_fn = CheckpointFnCreator(dirname=f"{output_dir}/saved_models", require_empty=False)

sourceAccuracyValidator = AccuracyValidator()
val_hooks = [ScoreHistory(sourceAccuracyValidator)]

trainer = Ignite(
    adapter, val_hooks=val_hooks, checkpoint_fn=checkpoint_fn, device=device
)
print(trainer.device)

early_stopper_kwargs = {"patience": args.patience}

start_time = datetime.now()

best_score, best_epoch = trainer.run(
    datasets, dataloader_creator=dc, max_epochs=args.max_epochs, early_stopper_kwargs=early_stopper_kwargs
)

end_time = datetime.now()
training_time = end_time - start_time

print(f"best_score={best_score}, best_epoch={best_epoch}")

plt.plot(val_hooks[0].score_history, label='source')
plt.title("val accuracy")
plt.legend()
plt.savefig(f"{output_dir}/val_accuracy.png")
plt.close('all')


cuda:0


[1/35]   3%|2         |it [00:00<?]

[1/9]  11%|#1        |it [00:00<?]

best_score=None, best_epoch=None


AttributeError: 'Namespace' object has no attribute 'dataroot'

In [ ]:

datasets = get_office31([source_domain], [target_domain], folder=args.data_root, return_target_with_labels=True)
dc = DataloaderCreator(batch_size=args.batch_size, num_workers=args.num_workers, all_val=True)

validator = AccuracyValidator(key_map={"src_val": "src_val"})
src_score = trainer.evaluate_best_model(datasets, validator, dc)
print("Source acc:", src_score)

validator = AccuracyValidator(key_map={"target_val_with_labels": "src_val"})
target_score = trainer.evaluate_best_model(datasets, validator, dc)
print("Target acc:", target_score)
print("---------")

[1/9]  11%|#1        |it [00:00<?]

Source acc: 0.868794322013855


[1/3]  33%|###3      |it [00:00<?]

Target acc: 0.74842768907547
---------


In [ ]:
C2 = copy.deepcopy(C)    

In [93]:
source_domain = 'amazon'
target_domain = 'webcam'
G = office31G(pretrained=False).to(device)
C = office31C(pretrained=False).to(device)


optimizers = Optimizers((torch.optim.Adam, {"lr": 1e-4}))
lr_schedulers = LRSchedulers((torch.optim.lr_scheduler.ExponentialLR, {"gamma": 0.99}))    

models = Models({"G": G, "C": C})
adapter= ClassifierAdapter(models=models, optimizers=optimizers, lr_schedulers=lr_schedulers)


output_dir = "tmp"
checkpoint_fn = CheckpointFnCreator(dirname=f"{output_dir}/saved_models", require_empty=False)

sourceAccuracyValidator = AccuracyValidator()
val_hooks = [ScoreHistory(sourceAccuracyValidator)]

new_trainer = Ignite(
    adapter, val_hooks=val_hooks, checkpoint_fn=checkpoint_fn, device=device
)
print(trainer.device)

from pytorch_adapt.frameworks.ignite import (
    CheckpointFnCreator,
    IgniteValHookWrapper,
    checkpoint_utils,
)

objs = [
        {
            "engine": new_trainer.trainer,
            "validator": new_trainer.validator,
            "val_hook0": val_hooks[0],
            **checkpoint_utils.adapter_to_dict(new_trainer.adapter),
        }
    ]
    
# best_score, best_epoch = trainer.run(
#     datasets, dataloader_creator=dc, max_epochs=args.max_epochs, early_stopper_kwargs=early_stopper_kwargs
# )

for to_load in objs:
    checkpoint_fn.load_best_checkpoint(to_load)



cuda:0


In [94]:

datasets = get_office31([source_domain], [target_domain], folder=args.data_root, return_target_with_labels=True)
dc = DataloaderCreator(batch_size=args.batch_size, num_workers=args.num_workers, all_val=True)

validator = AccuracyValidator(key_map={"src_val": "src_val"})
src_score = new_trainer.evaluate_best_model(datasets, validator, dc)
print("Source acc:", src_score)

validator = AccuracyValidator(key_map={"target_val_with_labels": "src_val"})
target_score = new_trainer.evaluate_best_model(datasets, validator, dc)
print("Target acc:", target_score)
print("---------")

[1/9]  11%|#1        |it [00:00<?]

Source acc: 0.868794322013855


[1/3]  33%|###3      |it [00:00<?]

Target acc: 0.74842768907547
---------


In [89]:

datasets = get_office31([source_domain], [target_domain],
                            folder=args.data_root,
                            return_target_with_labels=True,
                            download=args.download)
    
dc = DataloaderCreator(batch_size=args.batch_size,
                    num_workers=args.num_workers,
                    train_names=["train"],
                    val_names=["src_train", "target_train", "src_val", "target_val",
                                    "target_train_with_labels", "target_val_with_labels"])

G = new_trainer.adapter.models["G"]
C = new_trainer.adapter.models["C"]
D = Discriminator(in_size=2048, h=1024).to(device)

optimizers = Optimizers((torch.optim.Adam, {"lr": 0.001}))
lr_schedulers = LRSchedulers((torch.optim.lr_scheduler.ExponentialLR, {"gamma": 0.99}))
# lr_schedulers = LRSchedulers((torch.optim.lr_scheduler.MultiStepLR, {"milestones": [2, 5, 10, 20, 40], "gamma": hp.gamma}))

models = Models({"G": G, "C": C, "D": D})
adapter = DANN(models=models, optimizers=optimizers, lr_schedulers=lr_schedulers)


In [90]:

output_dir = "tmp"
checkpoint_fn = CheckpointFnCreator(dirname=f"{output_dir}/saved_models", require_empty=False)

sourceAccuracyValidator = AccuracyValidator()
targetAccuracyValidator = AccuracyValidator(key_map={"target_val_with_labels": "src_val"})
val_hooks = [ScoreHistory(sourceAccuracyValidator), ScoreHistory(targetAccuracyValidator)]

trainer = Ignite(
    adapter, val_hooks=val_hooks, device=device
)
print(trainer.device)

best_score, best_epoch = trainer.run(
    datasets, dataloader_creator=dc, max_epochs=args.max_epochs, early_stopper_kwargs=early_stopper_kwargs, check_initial_score=True
)


cuda:0


[1/3]  33%|###3      |it [00:00<?]

[1/9]  11%|#1        |it [00:00<?]

[1/9]  11%|#1        |it [00:00<?]

[1/3]  33%|###3      |it [00:00<?]

[1/9]  11%|#1        |it [00:00<?]

In [91]:
val_hooks[0]

ScoreHistory(
  validator=AccuracyValidator(required_data=['src_val'])
  latest_score=0.30319148302078247
  best_score=0.868794322013855
  best_epoch=0
)

In [92]:
val_hooks[1]

ScoreHistory(
  validator=AccuracyValidator(required_data=['target_val_with_labels'])
  latest_score=0.2515723407268524
  best_score=0.74842768907547
  best_epoch=0
)

In [86]:
del trainer

In [87]:
import gc
gc.collect()

21169

In [88]:
torch.cuda.empty_cache()

In [95]:
args.vishook_frequency = 133

In [96]:
args

Namespace(batch_size=64, data_root='./datasets/pytorch-adapt/', download=False, gamma=0.99, hp_tune=False, initial_trial=0, lr=0.0001, max_epochs=1, model_names=['DANN'], num_workers=1, patience=2, results_root='./results/', root='./', source=None, target=None, trials_count=1, vishook_frequency=133)

: 

-----

In [3]:
path = "/media/10TB71/shashemi/Domain-Adaptation/results/DAModels.CDAN/2000/a2d/saved_models"

In [5]:
source_domain = 'amazon'
target_domain = 'dslr'
G = office31G(pretrained=False).to(device)
C = office31C(pretrained=False).to(device)


optimizers = Optimizers((torch.optim.Adam, {"lr": 1e-4}))
lr_schedulers = LRSchedulers((torch.optim.lr_scheduler.ExponentialLR, {"gamma": 0.99}))    

models = Models({"G": G, "C": C})
adapter= ClassifierAdapter(models=models, optimizers=optimizers, lr_schedulers=lr_schedulers)


output_dir = "tmp"
checkpoint_fn = CheckpointFnCreator(dirname=f"{output_dir}/saved_models", require_empty=False)

sourceAccuracyValidator = AccuracyValidator()
val_hooks = [ScoreHistory(sourceAccuracyValidator)]

new_trainer = Ignite(
    adapter, val_hooks=val_hooks, checkpoint_fn=checkpoint_fn, device=device
)

from pytorch_adapt.frameworks.ignite import (
    CheckpointFnCreator,
    IgniteValHookWrapper,
    checkpoint_utils,
)

objs = [
        {
            "engine": new_trainer.trainer,
            "validator": new_trainer.validator,
            "val_hook0": val_hooks[0],
            **checkpoint_utils.adapter_to_dict(new_trainer.adapter),
        }
    ]
    
# best_score, best_epoch = trainer.run(
#     datasets, dataloader_creator=dc, max_epochs=args.max_epochs, early_stopper_kwargs=early_stopper_kwargs
# )

for to_load in objs:
    checkpoint_fn.load_best_checkpoint(to_load)



In [6]:

datasets = get_office31([source_domain], [target_domain], folder=args.data_root, return_target_with_labels=True)
dc = DataloaderCreator(batch_size=args.batch_size, num_workers=args.num_workers, all_val=True)

validator = AccuracyValidator(key_map={"src_val": "src_val"})
src_score = new_trainer.evaluate_best_model(datasets, validator, dc)
print("Source acc:", src_score)

validator = AccuracyValidator(key_map={"target_val_with_labels": "src_val"})
target_score = new_trainer.evaluate_best_model(datasets, validator, dc)
print("Target acc:", target_score)
print("---------")

[1/9]  11%|#1        |it [00:00<?]

Source acc: 0.868794322013855


[1/2]  50%|#####     |it [00:00<?]

Target acc: 0.7200000286102295
---------


In [10]:
source_domain = 'amazon'
target_domain = 'dslr'
G = new_trainer.adapter.models["G"]
C = new_trainer.adapter.models["C"]

G.fc = C.net[:6]
C.net = C.net[6:]


optimizers = Optimizers((torch.optim.Adam, {"lr": 1e-4}))
lr_schedulers = LRSchedulers((torch.optim.lr_scheduler.ExponentialLR, {"gamma": 0.99}))    

models = Models({"G": G, "C": C})
adapter= ClassifierAdapter(models=models, optimizers=optimizers, lr_schedulers=lr_schedulers)


output_dir = "tmp"
checkpoint_fn = CheckpointFnCreator(dirname=f"{output_dir}/saved_models", require_empty=False)

sourceAccuracyValidator = AccuracyValidator()
val_hooks = [ScoreHistory(sourceAccuracyValidator)]

more_new_trainer = Ignite(
    adapter, val_hooks=val_hooks, checkpoint_fn=checkpoint_fn, device=device
)

In [13]:
from pytorch_adapt.hooks import FeaturesAndLogitsHook

h1 = FeaturesAndLogitsHook()

In [19]:
h1(datasets)

KeyError: in FeaturesAndLogitsHook: __call__
in FeaturesHook: __call__
FeaturesHook: Getting src
FeaturesHook: Getting output: ['src_imgs_features']
FeaturesHook: Using model G with inputs: src_imgs
G